### **Importing the libraries**

In [1]:
!pip install python_speech_features
!pip install soundfile
!pip install librosa
import os
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed,BatchNormalization
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from python_speech_features import mfcc
import pickle
from keras.callbacks import ModelCheckpoint
import pathlib
import librosa
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=1278b71952f170c0d929d558d297850eee21d3284159130c3924ecbfdbd09bc8
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


Using TensorFlow backend.


### **Loading and extracting data**

In [2]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz

--2020-03-11 14:19:09--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2020-03-11 14:19:09--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb14d52050f666ef2ca62042aa8.dl.dropboxusercontent.com/cd/0/inline/AzspIKVpCwMX5_LEEwJ7Amh6tkD0keVJWkG4pIweQo-yfV6roDCJ4YG9EiKBBBe9QToT6o_C9s7DCc0pp7fHGYEAwWfljLnRyPmdgYUg4PE1vg/file# [following]
--2020-03-11 14:19:09--  https://ucb14d52050f666ef2ca62042aa8.dl.dropboxusercontent.com/cd/0/inline/AzspIKVpCwMX5_LEEwJ7Amh6tkD0keVJWkG4pIweQo-yfV6roDCJ4YG9EiKBBBe9QToT6o_C9s7DCc0pp7fHGYEAwWfljLnRyPmdgYUg4PE1vg/file
Resolving ucb14d

In [3]:
!tar -xvf /content/genres.tar.gz

genres/
genres/blues/
genres/blues/blues.00000.au
genres/blues/blues.00001.au
genres/blues/blues.00002.au
genres/blues/blues.00003.au
genres/blues/blues.00004.au
genres/blues/blues.00005.au
genres/blues/blues.00006.au
genres/blues/blues.00007.au
genres/blues/blues.00008.au
genres/blues/blues.00009.au
genres/blues/blues.00010.au
genres/blues/blues.00011.au
genres/blues/blues.00012.au
genres/blues/blues.00013.au
genres/blues/blues.00014.au
genres/blues/blues.00015.au
genres/blues/blues.00016.au
genres/blues/blues.00017.au
genres/blues/blues.00018.au
genres/blues/blues.00019.au
genres/blues/blues.00020.au
genres/blues/blues.00021.au
genres/blues/blues.00022.au
genres/blues/blues.00023.au
genres/blues/blues.00024.au
genres/blues/blues.00025.au
genres/blues/blues.00026.au
genres/blues/blues.00027.au
genres/blues/blues.00028.au
genres/blues/blues.00029.au
genres/blues/blues.00030.au
genres/blues/blues.00031.au
genres/blues/blues.00032.au
genres/blues/blues.00033.au
genres/blues/blues.00034.a

**Extracting the Spectogram for all the audio files**


In [5]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

**Writing data into CSV**

In [0]:
import csv

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
       
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [8]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00067.au,0.337922,0.129096,2258.538419,2176.031189,4755.429578,0.112765,-95.424423,101.368652,-20.682497,48.655476,-30.258639,20.802527,-18.303758,14.134917,-12.787580,13.433701,-14.072367,7.723721,-19.224149,2.856372,-9.809445,6.911756,-11.303164,8.940704,-2.959639,-0.975428,blues
1,blues.00072.au,0.306928,0.103914,2595.538006,2519.296119,5660.407439,0.129246,-118.665231,87.600960,6.563414,45.094287,-38.903451,20.815969,-26.852990,13.791710,-17.234161,5.104991,-15.705422,3.978413,-27.542309,3.050489,-15.946974,10.868560,-12.646966,2.820748,-13.337095,-4.375539,blues
2,blues.00093.au,0.377690,0.065906,569.930721,995.407125,927.427725,0.021701,-350.436204,169.545752,31.820372,16.682836,28.710147,19.256975,7.863395,10.036764,4.735106,2.907466,3.915553,1.821690,-5.970892,-5.259568,-0.229211,-1.776851,-3.713751,0.181590,2.072390,-2.896225,blues
3,blues.00036.au,0.233230,0.197818,1247.244815,1908.052722,2620.592487,0.036904,-200.220732,116.345181,18.060785,25.288819,0.315825,24.176778,-22.223080,11.183146,-26.841913,-7.643656,-24.872036,-12.015285,-17.009242,-8.115376,-11.959014,-12.617977,-14.187781,-8.204476,-15.024769,-10.322150,blues
4,blues.00020.au,0.302732,0.075387,1389.955510,1911.485152,3005.773491,0.052560,-230.413018,127.094186,7.151100,39.782090,1.951132,17.411689,-5.625576,8.252521,-18.105353,13.839343,-3.763585,2.561243,-4.329452,6.817760,-6.157659,-6.214758,-4.515434,-1.850600,-0.539056,1.508026,blues


In [9]:
data.shape

(1000, 28)

In [0]:
data = data.drop(['filename'],axis=1)

**Encoding the labels**

In [0]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

**Scaling the feature**

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

**Splitting data into train and test**

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
len(y_train)

800

In [15]:
len(y_test)

200

**Classification**

In [16]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





800/800 [==============================] - 1s 767us/step - loss: 2.1755 - acc: 0.2100
Epoch 2/100
800/800 [==============================] - 0s 37us/step - loss: 1.8574 - acc: 0.3712
Epoch 3/100
800/800 [==============================] - 0s 34us/step - loss: 1.6397 - acc: 0.4050
Epoch 4/100
800/800 [==============================] - 0s 35us/step - loss: 1.4566 - acc: 0.4850
Epoch 5/100
800/800 [==============================] - 0s 32us/step - loss: 1.3003 - acc: 0.5487
Epoch 6/100
800/800 [==============================] - 0s 33us/step - loss: 1.1917 - acc: 0.5850
Epoch 7/100
800/800 [==============================] - 0s 33us/step - loss: 1.0913 - acc: 0.6250
Epoch 8/100
800/800 [==============================] - 0s 36us/step - loss: 1.0179 - acc: 0.6512
Epoch 9/100
800/800 [==============================] - 0s 34us/step - loss: 0.9602 - acc: 0.6788
Epoch 10/100
800/800 [==

In [19]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 232us/step


In [20]:
print('test_acc: ',test_acc)

test_acc:  0.705


In [0]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [22]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=100,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/100
600/600 [==============================] - 0s 599us/step - loss: 2.3008 - acc: 0.1133 - val_loss: 2.2064 - val_acc: 0.2750
Epoch 2/100
600/600 [==============================] - 0s 57us/step - loss: 2.1491 - acc: 0.3767 - val_loss: 2.1074 - val_acc: 0.2350
Epoch 3/100
600/600 [==============================] - 0s 56us/step - loss: 2.0059 - acc: 0.3783 - val_loss: 2.0069 - val_acc: 0.2450
Epoch 4/100
600/600 [==============================] - 0s 55us/step - loss: 1.8538 - acc: 0.3900 - val_loss: 1.9142 - val_acc: 0.2650
Epoch 5/100
600/600 [==============================] - 0s 58us/step - loss: 1.7136 - acc: 0.4067 - val_loss: 1.8136 - val_acc: 0.2900
Epoch 6/100
600/600 [==============================] - 0s 61us/step - loss: 1.5893 - acc: 0.4383 - val_loss: 1.7360 - val_acc: 0.3200
Epoch 7/100
600/600 [==============================] - 0s 53us/step - loss: 1.4947 - acc: 0.4617 - val_loss: 1.6664 - val_acc: 0.3300
Epoch 8/100
600

In [23]:
results

[1.370507071018219, 0.675]

**Prediction**

In [0]:
predictions = model.predict(X_test)

In [25]:
predictions[0].shape

(10,)

In [26]:
np.sum(predictions[0])

0.99999994

In [27]:
np.argmax(predictions[0])

4